# 서울시 편의점 위치와 매출의 관계 분석

## 프로젝트 개요
- **목적**: 서울시 행정동별 편의점 매출에 영향을 미치는 요인 분석
- **기간**: 2022년 1분기 ~ 2025년 3분기
- **종속변수**: 행정동별 편의점 분기별 추정매출
- **독립변수**: 유동인구, 상권유형, 편의점 점포수(밀집도)

---

## 1. 환경 설정 및 데이터 로드

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

STORE_PATH = r'../data/processed/stores.csv'
SALES_PATH = r'../data/processed/sales.csv'
POPULATION_PATH = r'../data/processed/population.csv'
DISTRICTS_PATH = r'../data/processed/districts.csv'
# OUTPUT_DIR = r'C:/Users/Administrator/Desktop/자료모음/프로젝트_서울시_편의점_분석'

# STORE_PATH = r'C:/Users/Administrator/Desktop/자료모음/점포/0.서울시_상권분석서비스_점포_행정동_2022-2025_통합.csv'
# SALES_PATH = r'C:/Users/Administrator/Desktop/자료모음/추정매출/0.서울시_상권분석서비스_추정매출_행정동_2022-2025_통합.csv'
# POPULATION_PATH = r'C:/Users/Administrator/Desktop/자료모음/서울시 상권분석서비스(길단위인구-행정동).csv'
# DISTRICTS_PATH = r'C:/Users/Administrator/Desktop/자료모음/서울시 상권분석서비스(영역-상권).csv'
# OUTPUT_DIR = r'C:/Users/Administrator/Desktop/자료모음/프로젝트_서울시_편의점_분석'

store_df = pd.read_csv(STORE_PATH, encoding='utf-8-sig')
sales_df = pd.read_csv(SALES_PATH, encoding='utf-8-sig')
pop_df = pd.read_csv(POPULATION_PATH, encoding='utf-8-sig')
area_df = pd.read_csv(DISTRICTS_PATH, encoding='cp949')
# pop_df = pd.read_csv(POPULATION_PATH, encoding='cp949')
# area_df = pd.read_csv(DISTRICTS_PATH, encoding='cp949')

print(f'점포 데이터: {store_df.shape}')
print(f'매출 데이터: {sales_df.shape}')
print(f'유동인구 데이터: {pop_df.shape}')
print(f'상권영역 데이터: {area_df.shape}')

점포 데이터: (528758, 12)
매출 데이터: (256378, 53)
유동인구 데이터: (11475, 25)
상권영역 데이터: (1650, 11)


In [40]:
# ============================================================
# 인코딩 문제 수정
# ============================================================
# 문제: 행정동_코드_명에 '?' 문자가 포함되어 있음
#   예: '종로1?2?3?4가동' (잘못됨)
#   정상: '종로1·2·3·4가동' (올바름)
#
# 원인: 원본 CSV 파일 인코딩 과정에서 중점(·) → 물음표(?) 변환
#
# 영향: 병합(merge) 시 행정동_코드_명이 일치하지 않아 결측치 발생
#   - cvs_sales: '종로1·2·3·4가동'
#   - pop_df:    '종로1?2?3?4가동'
#   → 병합 실패 → 총_유동인구_수 결측
#
# 해결: 모든 '?'를 '·'로 변경

print("=" * 70)
print("1.5 데이터 전처리 (인코딩 문제 수정)")
print("=" * 70)

# 수정 전 확인
print("\n[수정 전 - '?' 포함 행 수]")
print(f"점포 데이터:     {store_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"매출 데이터:     {sales_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"유동인구 데이터: {pop_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"상권영역 데이터: {area_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")

# 예시 출력
print("\n[수정 전 예시]")
problem_examples = pop_df[pop_df['행정동_코드_명'].str.contains('?', regex=False)]['행정동_코드_명'].unique()[:5]
for example in problem_examples:
    print(f"  {example}")

1.5 데이터 전처리 (인코딩 문제 수정)

[수정 전 - '?' 포함 행 수]
점포 데이터:     1,743개
매출 데이터:     953개
유동인구 데이터: 189개
상권영역 데이터: 38개

[수정 전 예시]
  금호2?3가동
  면목3?8동
  중계2?3동
  상계3?4동
  상계6?7동


In [41]:
# ============================================================
# '?' → '·' 변경 실행
# ============================================================

print("\n수정 중...")

# 모든 데이터프레임의 행정동_코드_명 수정
store_df['행정동_코드_명'] = store_df['행정동_코드_명'].str.replace('?', '·')
sales_df['행정동_코드_명'] = sales_df['행정동_코드_명'].str.replace('?', '·')
pop_df['행정동_코드_명'] = pop_df['행정동_코드_명'].str.replace('?', '·')
area_df['행정동_코드_명'] = area_df['행정동_코드_명'].str.replace('?', '·')

print("✓ 수정 완료")


수정 중...
✓ 수정 완료


In [42]:
# ============================================================
# 수정 후 확인
# ============================================================

print("\n[수정 후 - '?' 포함 행 수]")
print(f"점포 데이터:     {store_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"매출 데이터:     {sales_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"유동인구 데이터: {pop_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")
print(f"상권영역 데이터: {area_df['행정동_코드_명'].str.contains('?', regex=False).sum():,}개")

# 예시 출력
print("\n[수정 후 예시]")
fixed_examples = pop_df[pop_df['행정동_코드_명'].str.contains('·')]['행정동_코드_명'].unique()[:5]
for example in fixed_examples:
    print(f"  {example}")

print("\n" + "=" * 70)
print("✓ 인코딩 문제 수정 완료")
print("  모든 '?'가 '·'로 변경되었습니다.")
print("  이제 병합 시 결측치가 대폭 감소할 것입니다.")
print("=" * 70)


[수정 후 - '?' 포함 행 수]
점포 데이터:     0개
매출 데이터:     0개
유동인구 데이터: 0개
상권영역 데이터: 0개

[수정 후 예시]
  금호2·3가동
  면목3·8동
  중계2·3동
  상계3·4동
  상계6·7동

✓ 인코딩 문제 수정 완료
  모든 '?'가 '·'로 변경되었습니다.
  이제 병합 시 결측치가 대폭 감소할 것입니다.


## 2. 데이터 기간 검증

In [43]:
print('='*60)
print('데이터 기간 검증 (목표: 2022년 1분기 ~ 2025년 3분기)')
print('='*60)

for name, df in [('점포', store_df), ('매출', sales_df), ('유동인구', pop_df)]:
    print(f"\n[{name} 데이터]")
    print(f"  기간: {df['기준_년분기_코드'].min()} ~ {df['기준_년분기_코드'].max()}")
    print(f"  분기 수: {df['기준_년분기_코드'].nunique()}개")

데이터 기간 검증 (목표: 2022년 1분기 ~ 2025년 3분기)

[점포 데이터]
  기간: 20221 ~ 20253
  분기 수: 15개

[매출 데이터]
  기간: 20221 ~ 20253
  분기 수: 15개

[유동인구 데이터]
  기간: 20191 ~ 20253
  분기 수: 27개


## 3. 데이터 전처리 및 통합

In [ ]:
# 편의점 데이터 필터링
cvs_store = store_df[store_df['서비스_업종_코드'] == 'CS300002'].copy()
cvs_sales = sales_df[sales_df['서비스_업종_코드'] == 'CS300002'].copy()
pop_df_filtered = pop_df[pop_df['기준_년분기_코드'] >= 20221].copy()

# 상권유형 매핑
area_type_map = {'A': '골목상권', 'D': '발달상권', 'R': '전통시장', 'U': '관광특구'}
area_df['상권유형'] = area_df['상권_구분_코드'].map(area_type_map)

# 상권유형 결정 함수: 관광특구 우선, 없으면 최빈값
def get_main_sangwon(sangwon_list):
    if '관광특구' in sangwon_list:
        return '관광특구'  # 관광특구가 있으면 최우선
    # 없으면 최빈값
    from collections import Counter
    return Counter(sangwon_list).most_common(1)[0][0]

area_by_dong = area_df.groupby('행정동_코드')['상권유형'].agg(list).reset_index()
area_by_dong['주요_상권유형'] = area_by_dong['상권유형'].apply(get_main_sangwon)
area_by_dong = area_by_dong[['행정동_코드', '주요_상권유형']]

dong_count_by_type = area_df.groupby('상권유형')['행정동_코드'].nunique().reset_index(name='동_개수')

print("="*60)
print("[상권유형별 행정동 수]")
print(dong_count_by_type)
print("="*60)

# 데이터 집계
cvs_store_agg = cvs_store.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '점포_수': 'sum',
        '프랜차이즈_점포_수': 'sum',
        '개업_점포_수': 'sum',
        '폐업_점포_수': 'sum'
    }
).reset_index()

cvs_sales_agg = cvs_sales.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '당월_매출_금액': 'sum',
        '당월_매출_건수': 'sum'
    }
).reset_index()

pop_agg = pop_df_filtered.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '총_유동인구_수': 'sum'
    }
).reset_index()

print(cvs_store_agg.describe())
print(cvs_sales_agg.describe())
print(pop_agg.describe())

# 데이터 통합
merged = cvs_sales_agg.merge(cvs_store_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(pop_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(area_by_dong, on='행정동_코드', how='left')
merged['연도'] = (merged['기준_년분기_코드'] // 10).astype(int) # 년도 추출
merged['분기'] = (merged['기준_년분기_코드'] % 10).astype(int) # 분기 추출

print(f'편의점 점포: {cvs_store.shape[0]:,}건')
print(f'편의점 매출: {cvs_sales.shape[0]:,}건')
print(f'통합 데이터: {merged.shape[0]:,}건 x {merged.shape[1]}컬럼')


   상권유형  동_개수
0  골목상권   381
1  관광특구     6
2  발달상권   155
3  전통시장   217
       기준_년분기_코드        행정동_코드     점포_수  프랜차이즈_점포_수  개업_점포_수  폐업_점포_수
count   6,375.00      6,375.00 6,375.00    6,375.00 6,375.00 6,375.00
mean   20,236.40 11,433,424.85     5.96       16.42     0.71     0.73
std        10.74    191,515.62     5.51       13.01     1.10     1.09
min    20,221.00 11,110,515.00     0.00        0.00     0.00     0.00
25%    20,224.00 11,260,655.00     3.00        9.00     0.00     0.00
50%    20,234.00 11,440,630.00     5.00       13.00     0.00     0.00
75%    20,244.00 11,590,680.00     8.00       20.00     1.00     1.00
max    20,253.00 11,740,700.00    51.00      111.00    11.00    11.00
       기준_년분기_코드        행정동_코드          당월_매출_금액     당월_매출_건수
count   6,097.00      6,097.00          6,097.00     6,097.00
mean   20,236.36 11,432,349.48  2,711,090,968.92   394,420.21
std        10.74    191,652.62  3,201,446,743.07   469,532.58
min    20,221.00 11,110,515.00        126,285.00    

In [45]:
area_by_dong = area_df.groupby('행정동_코드')['상권유형'].agg(
    lambda x: x.value_counts().index[0]
).reset_index()
area_by_dong.columns = ['행정동_코드', '주요_상권유형']

# 데이터 집계
cvs_store_agg = cvs_store.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '점포_수': 'sum',
        '프랜차이즈_점포_수': 'sum',
        '개업_점포_수': 'sum',
        '폐업_점포_수': 'sum'
    }
).reset_index()

cvs_sales_agg = cvs_sales.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '당월_매출_금액': 'sum',
        '당월_매출_건수': 'sum'
    }
).reset_index()

pop_agg = pop_df_filtered.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg(
    {
        '총_유동인구_수': 'sum'
    }
).reset_index()

print(cvs_store_agg.describe())
print(cvs_sales_agg.describe())
print(pop_agg.describe())

# 데이터 통합
merged = cvs_sales_agg.merge(cvs_store_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(pop_agg, on=['기준_년분기_코드', '행정동_코드', '행정동_코드_명'], how='left')
merged = merged.merge(area_by_dong, on='행정동_코드', how='left')
merged['연도'] = (merged['기준_년분기_코드'] // 10).astype(int) # 년도 추출
merged['분기'] = (merged['기준_년분기_코드'] % 10).astype(int) # 분기 추출

print(f'편의점 점포: {cvs_store.shape[0]:,}건')
print(f'편의점 매출: {cvs_sales.shape[0]:,}건')
print(f'통합 데이터: {merged.shape[0]:,}건 x {merged.shape[1]}컬럼')


       기준_년분기_코드        행정동_코드     점포_수  프랜차이즈_점포_수  개업_점포_수  폐업_점포_수
count   6,375.00      6,375.00 6,375.00    6,375.00 6,375.00 6,375.00
mean   20,236.40 11,433,424.85     5.96       16.42     0.71     0.73
std        10.74    191,515.62     5.51       13.01     1.10     1.09
min    20,221.00 11,110,515.00     0.00        0.00     0.00     0.00
25%    20,224.00 11,260,655.00     3.00        9.00     0.00     0.00
50%    20,234.00 11,440,630.00     5.00       13.00     0.00     0.00
75%    20,244.00 11,590,680.00     8.00       20.00     1.00     1.00
max    20,253.00 11,740,700.00    51.00      111.00    11.00    11.00
       기준_년분기_코드        행정동_코드          당월_매출_금액     당월_매출_건수
count   6,097.00      6,097.00          6,097.00     6,097.00
mean   20,236.36 11,432,349.48  2,711,090,968.92   394,420.21
std        10.74    191,652.62  3,201,446,743.07   469,532.58
min    20,221.00 11,110,515.00        126,285.00        22.00
25%    20,224.00 11,260,620.00    947,625,957.00   140,723.0

In [46]:

# ============================================================
# 상권유형 결측 처리
# ============================================================
# 원인: 24개 행정동이 상권영역(districts) 데이터에 없음
#   - 공식 상권으로 분류되지 않은 지역 (주로 주거지역)
#   - 예: 행당2동, 돈암1동, 월계3동 등
#
# 문제: 이 지역들도 편의점 매출이 있음 (평균 약 13억원)
#   - 삭제하면 339개 행(5.6%) 손실
#   - 유효한 데이터를 버리게 됨
#
# 해결: "미분류" 범주 생성
#   - 339개 행 데이터 보존
#   - 향후 회귀분석에서 더미변수로 활용 가능
#   - 상권 형성 여부가 매출에 미치는 영향 분석 가능

print("\n" + "=" * 70)
print("상권유형 결측 처리")
print("=" * 70)

# 결측 현황 확인
missing_count = merged['주요_상권유형'].isnull().sum()
missing_ratio = (missing_count / len(merged) * 100)

print(f"\n[처리 전]")
print(f"  주요_상권유형 결측: {missing_count}개 ({missing_ratio:.2f}%)")

# 결측이 발생하는 행정동 확인
missing_dongs = merged[merged['주요_상권유형'].isnull()]['행정동_코드_명'].unique()
print(f"  결측 발생 행정동: {len(missing_dongs)}개")
print(f"  예시: {', '.join(missing_dongs[:5])}")

# "미분류" 범주로 대체
merged['주요_상권유형'] = merged['주요_상권유형'].fillna('미분류')

print(f"\n[처리 후]")
print(f"  주요_상권유형 결측: {merged['주요_상권유형'].isnull().sum()}개")

print(f"\n[상권유형 분포]")
type_counts = merged['주요_상권유형'].value_counts()
for area_type, count in type_counts.items():
    ratio = (count / len(merged) * 100)
    print(f"  {area_type:8s}: {count:5,}개 ({ratio:5.2f}%)")

print("\n" + "=" * 70)
print("✓ 상권유형 결측 처리 완료")
print(f"  339개 행을 '미분류' 범주로 분류하여 데이터 보존")
print(f"  최종 분석 가능 데이터: {len(merged):,}건 (100% 보존)")
print("=" * 70)


상권유형 결측 처리

[처리 전]
  주요_상권유형 결측: 339개 (5.56%)
  결측 발생 행정동: 24개
  예시: 행당2동, 돈암1동, 월계3동, 하계2동, 중계1동

[처리 후]
  주요_상권유형 결측: 0개

[상권유형 분포]
  골목상권    : 5,182개 (84.99%)
  발달상권    :   345개 ( 5.66%)
  미분류     :   339개 ( 5.56%)
  전통시장    :   231개 ( 3.79%)

✓ 상권유형 결측 처리 완료
  339개 행을 '미분류' 범주로 분류하여 데이터 보존
  최종 분석 가능 데이터: 6,097건 (100% 보존)


In [47]:
merged.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,당월_매출_금액,당월_매출_건수,점포_수,프랜차이즈_점포_수,개업_점포_수,폐업_점포_수,총_유동인구_수,주요_상권유형,연도,분기
0,20221,11110515,청운효자동,"1,202,573,186.00",179263,4,5,0,0,3627519,골목상권,2022,1
1,20221,11110530,사직동,"3,761,850,443.00",659962,4,25,0,1,3402653,골목상권,2022,1
2,20221,11110540,삼청동,"1,226,802,003.00",138702,1,8,0,0,821735,발달상권,2022,1
3,20221,11110550,부암동,"607,056,495.00",79978,2,6,1,1,1283546,골목상권,2022,1
4,20221,11110560,평창동,"568,959,905.00",72916,3,14,1,0,823714,골목상권,2022,1


---
## 4. EDA (탐색적 데이터 분석)
### 4.1 기초 통계량

In [48]:
print('='*70)
print('기초 통계량')
print('='*70)
merged[['당월_매출_금액', '점포_수', '총_유동인구_수']].describe()

기초 통계량


,당월_매출_금액,점포_수,총_유동인구_수
count,"6,097.00","6,097.00","6,097.00"
mean,"2,711,090,968.92",6.16,"5,618,797.62"
std,"3,201,446,743.07",5.55,"2,968,712.52"
min,"126,285.00",0.00,"23,239.00"
25%,"947,625,957.00",3.00,"3,544,133.00"
50%,"1,719,118,340.00",5.00,"5,257,821.00"
75%,"3,234,061,370.00",8.00,"6,940,288.00"
max,"35,191,306,523.00",51.00,"22,328,990.00"


### 4.2 결측치 확인

In [49]:
print('결측치 현황:')
null_counts = merged.isnull().sum()
null_counts[null_counts > 0]

결측치 현황:


Series([], dtype: int64)

### 4.2.1 결측치 분석

In [50]:
# 목적: 전체 변수 중 어느 변수에 얼마나 결측치가 있는지 파악
# 중요성: 결측치 비율에 따라 처리 전략이 달라짐
# 5% 미만: 삭제 고려 가능
# 5~20%: 대체 방법 검토 필요
# 20% 이상: 변수 제외 또는 신중한 대체 필요

print("\n[전체 변수 결측치 현황]")
print(f"전체 데이터: {len(merged):,}건")
print(f"전체 변수: {len(merged.columns)}개\n")

# 결측치가 있는 변수만 출력
missing_summary = pd.DataFrame({
    '결측_개수': merged.isnull().sum(),
    '결측_비율(%)': (merged.isnull().sum() / len(merged) * 100).round(2)
})
missing_summary = missing_summary[missing_summary['결측_개수'] > 0].sort_values('결측_개수', ascending=False)

if len(missing_summary) > 0:
    print("결측치가 있는 변수:")
    for col in missing_summary.index:
        count = missing_summary.loc[col, '결측_개수']
        ratio = missing_summary.loc[col, '결측_비율(%)']
        print(f"  {col:20s}: {count:4.0f}개 ({ratio:6.2f}%)")
else:
    print("✓ 모든 변수에 결측치가 없습니다.")


[전체 변수 결측치 현황]
전체 데이터: 6,097건
전체 변수: 13개

✓ 모든 변수에 결측치가 없습니다.


In [51]:
# 분석 대상 변수의 결측치 집중 분석
# 향후 통계 분석에 사용할 주요 변수들만 선택
analysis_cols = ['당월_매출_금액', '당월_매출_건수', '점포_수', 
                 '총_유동인구_수', '주요_상권유형', '연도', '분기']

print("\n" + "=" * 70)
print("[분석 대상 변수의 결측치 현황]")
print("=" * 70)
print("※ 이 변수들은 향후 통계 분석에 사용될 가능성이 높은 핵심 변수입니다.\n")

analysis_missing = merged[analysis_cols].isnull().sum()
analysis_missing_ratio = (analysis_missing / len(merged) * 100).round(2)

for col in analysis_cols:
    count = analysis_missing[col]
    ratio = analysis_missing_ratio[col]
    status = "✓" if count == 0 else "⚠"
    print(f"{status} {col:20s}: {count:4.0f}개 ({ratio:6.2f}%)")


[분석 대상 변수의 결측치 현황]
※ 이 변수들은 향후 통계 분석에 사용될 가능성이 높은 핵심 변수입니다.

✓ 당월_매출_금액            :    0개 (  0.00%)
✓ 당월_매출_건수            :    0개 (  0.00%)
✓ 점포_수                :    0개 (  0.00%)
✓ 총_유동인구_수            :    0개 (  0.00%)
✓ 주요_상권유형             :    0개 (  0.00%)
✓ 연도                  :    0개 (  0.00%)
✓ 분기                  :    0개 (  0.00%)


In [52]:
# 총 결측 행 수 계산
# 하나라도 결측이 있는 행의 개수
total_missing_rows = merged[analysis_cols].isnull().any(axis=1).sum()
clean_rows = len(merged) - total_missing_rows

print("\n" + "-" * 70)
print(f"결측치가 없는 완전한 행: {clean_rows:,}개 ({clean_rows/len(merged)*100:.2f}%)")
print(f"결측치가 있는 행:       {total_missing_rows:,}개 ({total_missing_rows/len(merged)*100:.2f}%)")
print("-" * 70)

# 경고 메시지
if total_missing_rows > 0:
    print(f"\n⚠ 주의: 결측치를 단순 삭제(dropna)하면 {total_missing_rows:,}개 행({total_missing_rows/len(merged)*100:.1f}%)이 손실됩니다.")
    print("   → 다음 단계에서 결측치 패턴을 분석하여 적절한 처리 방법을 결정합니다.")


----------------------------------------------------------------------
결측치가 없는 완전한 행: 6,097개 (100.00%)
결측치가 있는 행:       0개 (0.00%)
----------------------------------------------------------------------


### 4.2.2. 결측치 패턴 분석

In [53]:
# ============================================================
# 4.2.2 결측치 패턴 분석
# ============================================================
# 목적: 결측치가 어떤 패턴으로 발생하는지 분석
# 중요성: 결측치 유형(MCAR/MAR/MNAR)을 판단하여 적절한 처리 방법 선택

print("=" * 70)
print("4.2.2 결측치 패턴 분석")
print("=" * 70)

# 주요 결측 변수 확인
missing_vars = missing_summary.index.tolist() if len(missing_summary) > 0 else []

if len(missing_vars) == 0:
    print("\n✓ 결측치가 없으므로 패턴 분석을 생략합니다.")
else:
    print(f"\n결측이 있는 변수: {len(missing_vars)}개")
    print(f"  {', '.join(missing_vars)}")

4.2.2 결측치 패턴 분석

✓ 결측치가 없으므로 패턴 분석을 생략합니다.


In [54]:
# ============================================================
# 결측치 중복 패턴 분석
# ============================================================
# 목적: 여러 변수가 동시에 결측인지, 독립적으로 결측인지 확인

print("\n" + "=" * 70)
print("[결측치 중복 패턴 분석]")
print("=" * 70)

if '총_유동인구_수' in missing_vars and '주요_상권유형' in missing_vars:
    # 두 변수 결측 패턴 분석
    both_null = merged[['총_유동인구_수', '주요_상권유형']].isnull().all(axis=1).sum()
    pop_only = merged['총_유동인구_수'].isnull().sum() - both_null
    area_only = merged['주요_상권유형'].isnull().sum() - both_null
    
    print(f"\n총_유동인구_수만 결측:        {pop_only:4d}개")
    print(f"주요_상권유형만 결측:         {area_only:4d}개")
    print(f"둘 다 결측 (동시 결측):       {both_null:4d}개")
    print(f"{'─' * 50}")
    print(f"총 결측 행:                   {pop_only + area_only + both_null:4d}개")
    
    # 해석
    if both_null == 0:
        print("\n✓ 해석: 두 변수의 결측이 독립적으로 발생 (동시 결측 없음)")
        print("  → 각 변수의 결측 원인이 다를 가능성 높음")
        print("  → 각 변수별로 다른 처리 방법 적용 가능")
    else:
        print(f"\n⚠ 해석: {both_null}개 행에서 동시 결측 발생")
        print("  → 같은 원인으로 결측이 발생했을 가능성")
        print("  → 동시 결측 행은 삭제를 우선 고려")


[결측치 중복 패턴 분석]


In [55]:
# ============================================================
# 총_유동인구_수 결측 행 분석
# ============================================================

if '총_유동인구_수' in missing_vars:
    pop_missing = merged[merged['총_유동인구_수'].isnull()]
    
    print(f"결측 행 수: {len(pop_missing):,}개 ({len(pop_missing)/len(merged)*100:.2f}%)")
    
    # 기술통계량
    print("\n[결측 행의 기술통계량]")
    stats_cols = ['당월_매출_금액', '점포_수']
    print(pop_missing[stats_cols].describe())

In [56]:
# ============================================================
# 총_유동인구_수 결측 vs 비결측 평균 비교
# ============================================================

if '총_유동인구_수' in missing_vars:
    pop_missing = merged[merged['총_유동인구_수'].isnull()]
    pop_non_missing = merged[merged['총_유동인구_수'].notnull()]
    
    print("[평균 비교: 결측 vs 비결측]")
    print()
    
    for col in ['당월_매출_금액', '점포_수']:
        missing_avg = pop_missing[col].mean()
        non_missing_avg = pop_non_missing[col].mean()
        diff_pct = abs(missing_avg - non_missing_avg) / non_missing_avg * 100
        
        print(f"{col}:")
        print(f"  결측 그룹 평균:   {missing_avg:>15,.0f}")
        print(f"  비결측 그룹 평균: {non_missing_avg:>15,.0f}")
        print(f"  차이:            {diff_pct:>15.1f}%")
        print()

In [57]:
# ============================================================
# 총_유동인구_수 결측이 많이 발생하는 행정동
# ============================================================

if '총_유동인구_수' in missing_vars:
    pop_missing = merged[merged['총_유동인구_수'].isnull()]
    
    print("[결측이 많이 발생하는 행정동 TOP 5]")
    print()
    
    top_missing_dongs = pop_missing['행정동_코드_명'].value_counts().head()
    for dong, count in top_missing_dongs.items():
        print(f"  {dong:20s}: {count:3d}건")
    
    print("\n[결측치 유형 추정]")
    print("  추정: MAR (Missing At Random)")
    print("  근거: 특정 (행정동, 분기) 조합에서 데이터 수집 누락")
    print("  처리 권장: 같은 행정동의 다른 분기 평균으로 대체")

In [58]:
# ============================================================
# 주요_상권유형 결측 행 분석
# ============================================================

if '주요_상권유형' in missing_vars:
    area_missing = merged[merged['주요_상권유형'].isnull()]
    
    print(f"결측 행 수: {len(area_missing):,}개 ({len(area_missing)/len(merged)*100:.2f}%)")
    
    # 기술통계량
    print("\n[결측 행의 기술통계량]")
    stats_cols = ['당월_매출_금액', '점포_수']
    print(area_missing[stats_cols].describe())
    
    # 중요 지표
    avg_sales = area_missing['당월_매출_금액'].mean()
    avg_stores = area_missing['점포_수'].mean()
    
    print(f"\n→ 결측 지역 평균 매출: {avg_sales:,.0f}원")
    print(f"→ 결측 지역 평균 점포수: {avg_stores:.2f}개")
    
    if avg_sales > 0 and avg_stores > 0:
        print("\n⚠ 중요: 결측 지역도 실제 상권 활동이 있음")
        print("   → 단순 삭제 시 유효한 데이터 손실")
        print("   → 새로운 범주('미분류', '주거지역' 등) 생성 권장")

In [59]:
# ============================================================
# 주요_상권유형 결측 vs 비결측 평균 비교
# ============================================================

if '주요_상권유형' in missing_vars:
    area_missing = merged[merged['주요_상권유형'].isnull()]
    area_non_missing = merged[merged['주요_상권유형'].notnull()]
    
    print("[평균 비교: 결측 vs 비결측]")
    print()
    
    for col in ['당월_매출_금액', '점포_수']:
        missing_avg = area_missing[col].mean()
        non_missing_avg = area_non_missing[col].mean()
        diff_pct = abs(missing_avg - non_missing_avg) / non_missing_avg * 100
        
        print(f"{col}:")
        print(f"  결측 그룹 평균:   {missing_avg:>15,.0f}")
        print(f"  비결측 그룹 평균: {non_missing_avg:>15,.0f}")
        print(f"  차이:            {diff_pct:>15.1f}%")
        
        if diff_pct > 20:
            print(f"  ⚠ 20% 이상 차이 → 삭제 시 표본 편향 위험 높음")
        print()

In [60]:
# ============================================================
# 주요_상권유형 결측이 많이 발생하는 행정동
# ============================================================

if '주요_상권유형' in missing_vars:
    area_missing = merged[merged['주요_상권유형'].isnull()]
    
    print("[결측이 많이 발생하는 행정동 TOP 10]")
    print()
    
    top_missing_dongs = area_missing['행정동_코드_명'].value_counts().head(10)
    for dong, count in top_missing_dongs.items():
        print(f"  {dong:20s}: {count:3d}건")
    
    print("\n[결측치 유형 추정]")
    print("  추정: MNAR (Missing Not At Random)")
    print("  근거: 특정 행정동이 공식 상권 분류 데이터에 없음")
    print("        결측 자체가 '비공식 상권' 또는 '주거지역'을 의미할 가능성")
    print("  처리 권장: '미분류' 또는 '주거지역' 범주 생성")

### 4.3 상권유형별 분포

In [61]:
print('상권유형별 분포:')
type_dist = merged['주요_상권유형'].value_counts()
for t, cnt in type_dist.items():
    print(f'  {t}: {cnt:,}건 ({cnt/len(merged)*100:.1f}%)')
print(f'  결측: {merged["주요_상권유형"].isnull().sum():,}건')

상권유형별 분포:
  골목상권: 5,182건 (85.0%)
  발달상권: 345건 (5.7%)
  미분류: 339건 (5.6%)
  전통시장: 231건 (3.8%)
  결측: 0건


### 4.4 이상치 분석 (IQR)

In [62]:
print('='*70)
print('이상치 분석 (IQR 방법)')
print('='*70)

for col in ['당월_매출_금액', '점포_수', '총_유동인구_수']:
    Q1 = merged[col].quantile(0.25)
    Q3 = merged[col].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    outliers = merged[(merged[col] < lower) | (merged[col] > upper)]
    print(f'\n[{col}]')
    print(f'  정상범위: {max(0,lower):,.0f} ~ {upper:,.0f}')
    print(f'  이상치: {len(outliers):,}건 ({len(outliers)/len(merged)*100:.2f}%)')

이상치 분석 (IQR 방법)

[당월_매출_금액]
  정상범위: 0 ~ 6,663,714,490
  이상치: 512건 (8.40%)

[점포_수]
  정상범위: 0 ~ 16
  이상치: 301건 (4.94%)

[총_유동인구_수]
  정상범위: 0 ~ 12,034,520
  이상치: 192건 (3.15%)


### 4.5 상관관계 분석

In [63]:
corr_cols = ['당월_매출_금액', '점포_수', '총_유동인구_수']
corr_matrix = merged[corr_cols].corr()

print('상관계수 매트릭스:')
print(corr_matrix.round(4))

print('\n해석:')
print(f'  매출 vs 점포수: {corr_matrix.loc["당월_매출_금액", "점포_수"]:.4f} (강한 양의 상관)')
print(f'  매출 vs 유동인구: {corr_matrix.loc["당월_매출_금액", "총_유동인구_수"]:.4f} (중간 양의 상관)')
print(f'  점포수 vs 유동인구: {corr_matrix.loc["점포_수", "총_유동인구_수"]:.4f} (중간 양의 상관)')

상관계수 매트릭스:
          당월_매출_금액  점포_수  총_유동인구_수
당월_매출_금액      1.00  0.81      0.50
점포_수          0.81  1.00      0.48
총_유동인구_수      0.50  0.48      1.00

해석:
  매출 vs 점포수: 0.8065 (강한 양의 상관)
  매출 vs 유동인구: 0.4991 (중간 양의 상관)
  점포수 vs 유동인구: 0.4848 (중간 양의 상관)


### 4.6 연도별 평균 현황

In [64]:
yearly = merged.groupby('연도').agg({
    '당월_매출_금액': 'mean',
    '점포_수': 'mean',
    '총_유동인구_수': 'mean'
}).round(0)

print('연도별 평균 현황:')
for year in yearly.index:
    print(f'\n[{year}년]')
    print(f'  평균 매출: {yearly.loc[year, "당월_매출_금액"]:,.0f}원')
    print(f'  평균 점포수: {yearly.loc[year, "점포_수"]:.2f}개')
    print(f'  평균 유동인구: {yearly.loc[year, "총_유동인구_수"]:,.0f}명')

연도별 평균 현황:

[2022년]
  평균 매출: 2,748,484,308원
  평균 점포수: 6.00개
  평균 유동인구: 5,708,858명

[2023년]
  평균 매출: 2,785,407,949원
  평균 점포수: 6.00개
  평균 유동인구: 5,646,649명

[2024년]
  평균 매출: 2,694,249,705원
  평균 점포수: 6.00개
  평균 유동인구: 5,585,284명

[2025년]
  평균 매출: 2,583,634,058원
  평균 점포수: 6.00개
  평균 유동인구: 5,504,944명


### 4.7 상권유형별 매출 현황

In [65]:
type_stats = merged.groupby('주요_상권유형').agg({
    '당월_매출_금액': ['mean', 'median', 'std'],
    '점포_수': 'mean',
    '총_유동인구_수': 'mean'
}).round(0)

print('상권유형별 매출 현황:')
for t in type_stats.index:
    if pd.notna(t):
        print(f'\n[{t}]')
        print(f'  평균 매출: {type_stats.loc[t, ("당월_매출_금액", "mean")]:,.0f}원')
        print(f'  중앙값 매출: {type_stats.loc[t, ("당월_매출_금액", "median")]:,.0f}원')
        print(f'  평균 점포수: {type_stats.loc[t, ("점포_수", "mean")]:.1f}개')

상권유형별 매출 현황:

[골목상권]
  평균 매출: 2,537,951,651원
  중앙값 매출: 1,686,102,469원
  평균 점포수: 6.0개

[미분류]
  평균 매출: 1,328,236,688원
  중앙값 매출: 621,705,985원
  평균 점포수: 3.0개

[발달상권]
  평균 매출: 6,575,520,394원
  중앙값 매출: 5,518,513,166원
  평균 점포수: 12.0개

[전통시장]
  평균 매출: 2,852,941,173원
  중앙값 매출: 1,955,967,551원
  평균 점포수: 4.0개


In [66]:
#통합 데이터 저장
merged.to_csv('./통합_데이터.csv', encoding='utf-8-sig', index=False)
print(f'통합 데이터 저장 완료: {merged.shape}')

통합 데이터 저장 완료: (6097, 13)


In [67]:
# 기간 검증
print(f"분기 범위: {merged['기준_년분기_코드'].min()} ~ {merged['기준_년분기_코드'].max()}")
print(f"분기 수: {merged['기준_년분기_코드'].nunique()}개")
print(f"분기별 목록: {sorted(merged['기준_년분기_코드'].unique())}")

# 행정동 수 검증
print(f"행정동 수: {merged['행정동_코드'].nunique()}개")

# 분기별 행정동 수 확인
print(merged.groupby('기준_년분기_코드')['행정동_코드'].nunique())

분기 범위: 20221 ~ 20253
분기 수: 15개
분기별 목록: [np.int64(20221), np.int64(20222), np.int64(20223), np.int64(20224), np.int64(20231), np.int64(20232), np.int64(20233), np.int64(20234), np.int64(20241), np.int64(20242), np.int64(20243), np.int64(20244), np.int64(20251), np.int64(20252), np.int64(20253)]
행정동 수: 414개
기준_년분기_코드
20221    409
20222    408
20223    409
20224    409
20231    408
20232    407
20233    407
20234    406
20241    406
20242    405
20243    404
20244    404
20251    404
20252    404
20253    407
Name: 행정동_코드, dtype: int64


In [68]:
# 셀 1
print('=' * 70)
print('통합 데이터 설명')
print('=' * 70)
print('\n[1] 데이터 정의')
print('    서울시 행정동별 편의점 분기별 집계 데이터')
print('    - 분석 단위: 행정동 × 분기')
print(f'    - 총 Row 수: {merged.shape[0]:,}건')
print(f'    - 총 컬럼 수: {merged.shape[1]}개')

통합 데이터 설명

[1] 데이터 정의
    서울시 행정동별 편의점 분기별 집계 데이터
    - 분석 단위: 행정동 × 분기
    - 총 Row 수: 6,097건
    - 총 컬럼 수: 13개


In [69]:
# 셀 2
print('[2] 기간 검증')
print(f'    - 분기 범위: {merged["기준_년분기_코드"].min()} ~ {merged["기준_년분기_코드"].max()}')
print(f'    - 분기 수: {merged["기준_년분기_코드"].nunique()}개')
print(f'    - 분기 목록: {sorted(merged["기준_년분기_코드"].unique())}')

[2] 기간 검증
    - 분기 범위: 20221 ~ 20253
    - 분기 수: 15개
    - 분기 목록: [np.int64(20221), np.int64(20222), np.int64(20223), np.int64(20224), np.int64(20231), np.int64(20232), np.int64(20233), np.int64(20234), np.int64(20241), np.int64(20242), np.int64(20243), np.int64(20244), np.int64(20251), np.int64(20252), np.int64(20253)]


In [70]:
# 셀 3
print('[3] 행정동 검증')
print(f'    - 행정동 수: {merged["행정동_코드"].nunique()}개')
print(f'    - 계산: {merged.shape[0]:,}건 ÷ {merged["기준_년분기_코드"].nunique()}분기 = 약 {merged.shape[0] // merged["기준_년분기_코드"].nunique()}개 행정동/분기')

[3] 행정동 검증
    - 행정동 수: 414개
    - 계산: 6,097건 ÷ 15분기 = 약 406개 행정동/분기


In [71]:
# 셀 4
print('[4] 원본 데이터 → 통합 데이터 변환 과정')
print('    ┌─────────────────────────────────────────────────────────────┐')
print('    │ 원본 데이터                      │ 통합 데이터에 포함된 정보  │')
print('    ├─────────────────────────────────────────────────────────────┤')
print('    │ sales.csv (매출)                 │ 당월_매출_금액, 당월_매출_건수 │')
print('    │ stores.csv (점포)                │ 점포_수, 프랜차이즈_점포_수 등 │')
print('    │ population.csv (유동인구)        │ 총_유동인구_수              │')
print('    │ districts.csv (상권영역)         │ 주요_상권유형               │')
print('    └─────────────────────────────────────────────────────────────┘')

[4] 원본 데이터 → 통합 데이터 변환 과정
    ┌─────────────────────────────────────────────────────────────┐
    │ 원본 데이터                      │ 통합 데이터에 포함된 정보  │
    ├─────────────────────────────────────────────────────────────┤
    │ sales.csv (매출)                 │ 당월_매출_금액, 당월_매출_건수 │
    │ stores.csv (점포)                │ 점포_수, 프랜차이즈_점포_수 등 │
    │ population.csv (유동인구)        │ 총_유동인구_수              │
    │ districts.csv (상권영역)         │ 주요_상권유형               │
    └─────────────────────────────────────────────────────────────┘


In [72]:
# 셀 5
print('[5] 집계 방식')
print('    원본 (수십만 건) → 편의점 필터링 → 행정동+분기별 GROUP BY → 4개 데이터 MERGE')

[5] 집계 방식
    원본 (수십만 건) → 편의점 필터링 → 행정동+분기별 GROUP BY → 4개 데이터 MERGE


In [73]:
# 셀 6
print('[6] 분기별 데이터 건수')
quarterly_counts = merged.groupby('기준_년분기_코드')['행정동_코드'].nunique()
for q, cnt in quarterly_counts.items():
    year = q // 10
    quarter = q % 10
    print(f'    {year}년 {quarter}분기: {cnt}개 행정동')

[6] 분기별 데이터 건수
    2022년 1분기: 409개 행정동
    2022년 2분기: 408개 행정동
    2022년 3분기: 409개 행정동
    2022년 4분기: 409개 행정동
    2023년 1분기: 408개 행정동
    2023년 2분기: 407개 행정동
    2023년 3분기: 407개 행정동
    2023년 4분기: 406개 행정동
    2024년 1분기: 406개 행정동
    2024년 2분기: 405개 행정동
    2024년 3분기: 404개 행정동
    2024년 4분기: 404개 행정동
    2025년 1분기: 404개 행정동
    2025년 2분기: 404개 행정동
    2025년 3분기: 407개 행정동


In [74]:
# 셀 7
print('=' * 70)
print('결론: 통합 데이터 하나로 시각화 및 심층분석 수행 가능')
print('=' * 70)

결론: 통합 데이터 하나로 시각화 및 심층분석 수행 가능
